# Sweep $Q$ and fusion/fission to explore optimal quality control regimes

For submission on the Impeial HPC. Fix the magmitude of the fission rate and vary over the fusion rate by orders of magnitude. For each of these, look at different quality control strengths $Q$. 

Author: Juvid Aryaman

Date: 11/10/17

In [1]:
!pwd

/home/juvid/Dropbox/Work/Mit_and_Metabolism/Networks_mtDNA_dynamics/Stochastic_sim/Closed_loop_control/Singleton_birth_fus/const_fus/Param_sweeps/QC/sel_deg/Q_fs_sweep


In [2]:
work_dir = '/work/ja1109/networks/sing_birth_fus/param_sweep/QC_fs_sweep/SD'

In [3]:
import numpy as np
import pandas as pd
import sing_birth_fus_utils_ap as utls
import pdb

%reload_ext autoreload
%autoreload 2

Paramter ordering convention: h_init_mean, n_target, xi, beta, gamma, kappa, b, mu, delta, Q

In [4]:
nominal_param = (0.3,
1000, 
0.0,
33.120000000000005,
0.03785142857142858,
11.662903457629223,
1.2416523075924095e-05,
0.023,
1.0,
1.0)
h_init_mean, n_target, xi, beta, gamma, kappa, b, mu, delta, Q = nominal_param

In [5]:
np.hstack(([0],[1,2]))

array([0, 1, 2])

In [6]:
epsilon_space = np.linspace(-5,1,13)
Q_space = 1.0 + 10**epsilon_space
Q_space = np.hstack(([1], Q_space))
Q_space

array([  1.        ,   1.00001   ,   1.00003162,   1.0001    ,
         1.00031623,   1.001     ,   1.00316228,   1.01      ,
         1.03162278,   1.1       ,   1.31622777,   2.        ,
         4.16227766,  11.        ])

In [7]:
n_points = 21

lg_ls_low_gamma = -2
lg_ls_high_gamma = 2

gamma_space = 10**np.linspace(lg_ls_low_gamma,lg_ls_high_gamma,n_points) # gamma space

np.log10(gamma_space)

array([-2. , -1.8, -1.6, -1.4, -1.2, -1. , -0.8, -0.6, -0.4, -0.2,  0. ,
        0.2,  0.4,  0.6,  0.8,  1. ,  1.2,  1.4,  1.6,  1.8,  2. ])

In [8]:
n_iters_per_block = 1000 # multiply by NREPEATS in the .c script for the total number of iterations
#n_iters_per_block = 5

In [9]:
initial_seed = 66020 #np.round(np.random.uniform()*1e6).astype(int)
print(initial_seed)

66020


Create first set of parameter sweeps over ($\beta$,$\gamma$) at constant ($\kappa$,$b$).

In [10]:
time_per_job = '02:00:00' # Based on max churn taking 20s for 10 days. For 500 days, 10 iters and x10 safety

In [11]:
def make_submission_string(initial_seed, param_block, n_iters_per_block, new_param):
    submission_string = './closed_loop_deg_single_SD.ce {0} {1} {2} $PBS_ARRAY_INDEX'.format(initial_seed,
                                                                                             param_block,
                                                                                             n_iters_per_block)     
    for p in new_param:
        submission_string+=' {}'.format(p)
    
    submission_string+='\n'

    return submission_string

In [12]:
def make_submission_script(param_block,time_per_job,n_iters_per_block,initial_seed,new_param):
    f = open('submission_{}.pbs'.format(param_block),'w')
    f.write('#!/bin/sh\n')
    f.write('#PBS -N CLC_Q_fs_SD_{}\n'.format(param_block))
    f.write('#PBS -l walltime={}\n'.format(time_per_job))
    f.write('#PBS -l select=1:ncpus=1:mem=1gb\n')
    f.write('#PBS -J 0-{}\n'.format(n_iters_per_block - 1))
    
    f.write('cp $PBS_O_WORKDIR/closed_loop_deg_single_SD.ce $TMPDIR\n')
    
    submission_string = make_submission_string(initial_seed, param_block, n_iters_per_block, new_param)
      
    f.write(submission_string) 
    f.write('cp output_{0}_$PBS_ARRAY_INDEX.txt $PBS_O_WORKDIR/p{0:03d}\n'.format(param_block))
    
    # clean up -- have a file number quota
    f.write('sleep 10\n')
    f.write('rm $PBS_O_WORKDIR/CLC_Q_fs_SD_*\n')
    f.close()

Define a space over which to search $\kappa$ so we can find the steady state copy number

In [13]:
kappa_space = np.linspace(-1000,5000,6001)
kappa_space[:5]

array([-1000.,  -999.,  -998.,  -997.,  -996.])

In [14]:
param_cols = ['h_init_mean', 'n_target', 'xi', 'beta', 'gamma', 'kappa', 'b', 'mu', 'delta', 'Q', 
              'net_rat']

In [15]:
param_df = pd.DataFrame(columns=param_cols)

In [16]:
master_script = open('master.sh','w')
for i, gamma_change in enumerate(gamma_space):
    for j, newQ in enumerate(Q_space):
        submit_flag = False # assume we should not submit this parametrization   
        param_block = len(Q_space)*i+j
        
        # Update params
        newgamma = gamma * gamma_change 
        
        new_param = [k for k in nominal_param]
        new_param[4] = newgamma
        new_param[-1] = newQ
        
        # Find kappa for copy number when there is no QC
        res = utls.residual_n_manual(n_target, 250, xi, newgamma, beta, kappa_space, b, mu, delta, 1000.0)
        if np.min(res) < 0:
            index = np.argmin(res)
            newkappa = kappa_space[index]
            new_param[5] = newkappa
            
            submit_flag = True
            
            #             # The proposed value of kappa is ok
            #             new_param[5] = newkappa
            #             submit_flag = True

        else:
            print(param_block,i,j,np.min(res), kappa_space[np.argmin(res)],'no kappa')            
            
        
        param_df = param_df.append(pd.DataFrame([np.hstack((new_param,(gamma_change)))],columns=param_cols))
        
        if submit_flag:
            make_submission_script(param_block,time_per_job,n_iters_per_block,initial_seed,new_param)

            master_script.write('echo "{}"\n'.format(param_block))
            master_script.write('mkdir -p {0}/p{1:03d}\n'.format(work_dir,param_block))
            master_script.write("qsub submission_{}.pbs\n".format(param_block))
            master_script.write('sleep 0.5\n')
            print(param_block,i,j)
        else:
            s = make_submission_string(initial_seed, param_block, n_iters_per_block, new_param)
            print(s)
            
master_script.close()
param_df.reset_index(inplace=True, drop=True)        

(0, 0, 0)
(1, 0, 1)
(2, 0, 2)
(3, 0, 3)
(4, 0, 4)
(5, 0, 5)
(6, 0, 6)
(7, 0, 7)
(8, 0, 8)
(9, 0, 9)
(10, 0, 10)
(11, 0, 11)
(12, 0, 12)
(13, 0, 13)
(14, 1, 0)
(15, 1, 1)
(16, 1, 2)
(17, 1, 3)
(18, 1, 4)
(19, 1, 5)
(20, 1, 6)
(21, 1, 7)
(22, 1, 8)
(23, 1, 9)
(24, 1, 10)
(25, 1, 11)
(26, 1, 12)
(27, 1, 13)
(28, 2, 0)
(29, 2, 1)
(30, 2, 2)
(31, 2, 3)
(32, 2, 4)
(33, 2, 5)
(34, 2, 6)
(35, 2, 7)
(36, 2, 8)
(37, 2, 9)
(38, 2, 10)
(39, 2, 11)
(40, 2, 12)
(41, 2, 13)
(42, 3, 0)
(43, 3, 1)
(44, 3, 2)
(45, 3, 3)
(46, 3, 4)
(47, 3, 5)
(48, 3, 6)
(49, 3, 7)
(50, 3, 8)
(51, 3, 9)
(52, 3, 10)
(53, 3, 11)
(54, 3, 12)
(55, 3, 13)
(56, 4, 0)
(57, 4, 1)
(58, 4, 2)
(59, 4, 3)
(60, 4, 4)
(61, 4, 5)
(62, 4, 6)
(63, 4, 7)
(64, 4, 8)
(65, 4, 9)
(66, 4, 10)
(67, 4, 11)
(68, 4, 12)
(69, 4, 13)
(70, 5, 0)
(71, 5, 1)
(72, 5, 2)
(73, 5, 3)
(74, 5, 4)
(75, 5, 5)
(76, 5, 6)
(77, 5, 7)
(78, 5, 8)
(79, 5, 9)
(80, 5, 10)
(81, 5, 11)
(82, 5, 12)
(83, 5, 13)
(84, 6, 0)
(85, 6, 1)
(86, 6, 2)
(87, 6, 3)


sing_birth_fus_utils_ap.py:69: RuntimeWarning: invalid value encountered in sqrt
  b**2*(beta**2 - 4*(-1 + delta)*gamma*ms*mu*(-1 + xi) + 2*beta*(gamma*kappa + 3*mu - mu*xi) + (gamma*kappa + mu + mu*xi)**2)))/\
sing_birth_fus_utils_ap.py:73: RuntimeWarning: invalid value encountered in sqrt
  b**2*(beta**2 - 4*(-1 + delta)*gamma*ms*mu*(-1 + xi) + 2*beta*(gamma*kappa + 3*mu - mu*xi) + (gamma*kappa + mu + mu*xi)**2))
sing_birth_fus_utils_ap.py:88: RuntimeWarning: invalid value encountered in sqrt
  b**2*(beta**2 - 4*(-1 + delta)*gamma*ms*mu*(-1 + xi) + 2*beta*(gamma*kappa + 3*mu - mu*xi) + (gamma*kappa + mu + mu*xi)**2))\


(88, 6, 4)
(89, 6, 5)
(90, 6, 6)
(91, 6, 7)
(92, 6, 8)
(93, 6, 9)
(94, 6, 10)
(95, 6, 11)
(96, 6, 12)
(97, 6, 13)
(98, 7, 0)
(99, 7, 1)
(100, 7, 2)
(101, 7, 3)
(102, 7, 4)
(103, 7, 5)
(104, 7, 6)
(105, 7, 7)
(106, 7, 8)
(107, 7, 9)
(108, 7, 10)
(109, 7, 11)
(110, 7, 12)
(111, 7, 13)
(112, 8, 0)
(113, 8, 1)
(114, 8, 2)
(115, 8, 3)
(116, 8, 4)
(117, 8, 5)
(118, 8, 6)
(119, 8, 7)
(120, 8, 8)
(121, 8, 9)
(122, 8, 10)
(123, 8, 11)
(124, 8, 12)
(125, 8, 13)
(126, 9, 0)
(127, 9, 1)
(128, 9, 2)
(129, 9, 3)
(130, 9, 4)
(131, 9, 5)
(132, 9, 6)
(133, 9, 7)
(134, 9, 8)
(135, 9, 9)
(136, 9, 10)
(137, 9, 11)
(138, 9, 12)
(139, 9, 13)
(140, 10, 0)
(141, 10, 1)
(142, 10, 2)
(143, 10, 3)
(144, 10, 4)
(145, 10, 5)
(146, 10, 6)
(147, 10, 7)
(148, 10, 8)
(149, 10, 9)
(150, 10, 10)
(151, 10, 11)
(152, 10, 12)
(153, 10, 13)
(154, 11, 0)
(155, 11, 1)
(156, 11, 2)
(157, 11, 3)
(158, 11, 4)
(159, 11, 5)
(160, 11, 6)
(161, 11, 7)
(162, 11, 8)
(163, 11, 9)
(164, 11, 10)
(165, 11, 11)
(166, 11, 12)
(167, 11, 13)


In [17]:
param_df.to_csv('param_sweep_vals_SD.csv',index=False)